Building likelihood maps for a set of representative images.

> TODO: use images from dataset and superimpose the bounding boxes on them

In [ ]:
import retinoto_py as fovea
args = fovea.Params()
args

In [ ]:
import torch
from torchvision.io import read_image

alpha = .5
s_max = 1000

In [ ]:
idx_to_label = fovea.get_idx_to_label(args)
label2idx = fovea.get_label_to_idx(args)
label2idx['impala']

In [ ]:
for key in label2idx.keys(): 
    if 'owl' in key: print(key)

In [ ]:
dataset = 'bbox'
model_filename = args.data_cache / f'33_fovea_model_name={args.model_name}_dataset={dataset}.pth'
model = fovea.load_model(args, model_filename=model_filename)
model_filename

In [ ]:
from torchvision.transforms.functional import InterpolationMode, resize
image_size_full = 1024

for image_url, true_label in  [('./images/jaguar_5.jpg',  'jaguar'),
                                ('./images/frog.jpg',  'tree_frog'),
                                ('./images/Hidden_snow_leopard.png',  'snow_leopard'),
                                ('./images/Hidden_leopard.png',  'leopard'),
                                ('./images/Hidden_giraffe.png',  'giraffe'),
                                ('./images/Hidden_tiger.png',  'tiger'),
                                ('./images/Hidden_owl.png',  'great_grey_owl'),
                                ('./images/wolf.jpg',  'wolf'),
                                ]:
    full_image = read_image(image_url)[:3, :, :]/255
    full_image = resize(full_image, image_size_full, interpolation=InterpolationMode.BILINEAR, antialias=True)
    print(f"{type(full_image) = }, {full_image.dtype = }, {full_image.shape = }")

    for do_fovea  in [True, False]:
        args = fovea.Params(batch_size=N_show, do_mask=not(do_fovea), do_fovea=do_fovea)

        pos_H, pos_W, outputs = fovea.compute_likelihood_map(args, model, full_image, size_ratio=.6)
        outputs = outputs.cpu()
        logit_label = outputs[:, label2idx[true_label]]
        idx_max = logit_label.argmax()


        fig, ax = fovea.plt.subplots()
        full_image_np = torch.movedim(full_image, (1, 2, 0), (0, 1, 2)).numpy()
        ax.imshow(full_image_np)
        ax.scatter(pos_W.ravel(), pos_H.ravel(), s=logit_label.abs()*s_max, c=logit_label, alpha=alpha, edgecolors='none', cmap='coolwarm', vmin=-max(abs(logit_label)), vmax=max(abs(logit_label)),)

        ax.scatter(pos_W.ravel()[idx_max], pos_H.ravel()[idx_max], s=logit_label[idx_max]*s_max, marker='*', c='white', alpha=alpha)
        fig.set_facecolor(color='white')
        fovea.plt.show()